# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [4]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
D = 11
T = 50

OTHER_DATE = '2024-03-24'
DOUBLE_MSMT = True
BEST_PATH = False

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, D, T, DEVICE, _is_hex, OTHER_DATE, DOUBLE_MSMT, best_path=BEST_PATH)

10:14:52 Warning: Negative T2 error -0.08 % for qubit 38, setting to 0. T1: 6.421500990479847e-05, T2: 0.00015673252358240422. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 54
10:14:52 Warning: Negative T2 error -0.01 % for qubit 39, setting to 0. T1: 0.00013169172467058469, T2: 0.00027539403593426396. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 54
10:14:52 Warning: Negative T2 error -0.01 % for qubit 111, setting to 0. T1: 0.00016588647661947383, T2: 0.00034179903119965343. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 54
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 17:49:53.348518+00:00.


# Needed decoding objects

In [7]:
import numpy as np
import cpp_soft_info as csi
from soft_info import inv_qubit_mapping

relError = 1
bandwidths = [0.6]
num_points = 20

kde_dict_PS = csi.get_KDEs(simulator.all_memories, bandwidths, relError=relError, absError=-1, num_points = num_points)
inverted_q_map = inv_qubit_mapping(simulator.qubit_mapping)

# Loop function

In [ ]:
from tqdm import tqdm
import json
import cpp_soft_info
import pymatching
from soft_info import RepetitionCodeStimCircuit

def funct(distance, rounds, xbasis, resets=False, shots=1e5):
    # No logical bcs symm noise model
    simulator = RepCodeIQSimulator(provider, distance, rounds, DEVICE, _is_hex, OTHER_DATE, DOUBLE_MSMT, best_path=BEST_PATH)
    IQ_data = simulator.generate_IQ(shots, logical='0', xbasis=xbasis, resets=resets, individual=False)

    # Get KDEs
    pSoft_PS, countMat_PS = csi.iqConvertorEstim(IQ_data, inverted_q_map, kde_dict_PS, relError, -1)

    # stim models
    circ_soft = RepetitionCodeStimCircuit(distance, rounds, xbasis, resets, noise_list=simulator.noise_list, 
                                          subsampling=False, no_fin_soft=True)
    model_soft = circ_soft.circuits['0'].detector_error_model()

    used_msmt_err_dict = {key: value for key, value in simulator.msmt_err_dict.items() if key in simulator.layout}
    ps_mean = sum(value['p_soft'] for value in used_msmt_err_dict.values()) / len(used_msmt_err_dict)
    noise_list_h = simulator.noise_list.copy()
    noise_list_h[-1]=ps_mean
    circ_hard = RepetitionCodeStimCircuit(distance, rounds, xbasis, resets, noise_list=simulator.noise_list,
                                            subsampling=False, no_fin_soft=False)
    model_hard = circ_hard.circuits['0'].detector_error_model()


    # Decode
    res_s = csi.decodeConvertorAll(model_soft, countMat_PS, pSoft_PS, rounds, 0, resets, decode_hard=False)
    res_h = csi.decodeConvertorAll(model_hard, countMat_PS, pSoft_PS, rounds, 0, resets, decode_hard=True)

    return res_s, res_h

In [4]:
from tqdm import tqdm
import json
import cpp_soft_info
import pymatching

def func(distance, rounds, resets=False, shots = 1e5, p_meas = -1):
    # Initialize simulator
    simulator = RepCodeIQSimulator(provider, distance, rounds, DEVICE, _is_hex=_is_hex, _resets = resets, other_date=OTHER_DATE)
    IQ_data= simulator.generate_IQ(int(shots), noise_list=NOISE_LIST, verbose=False)
    print("Generated IQ data")

    # Run decoding
    result_grid = cpp_soft_info.decode_IQ_fast(simulator.stim_circ.detector_error_model(decompose_errors=False), IQ_data,
                                        rounds, int(LOGICAL), resets, simulator.qubit_mapping, simulator.grid_dict,
                                        simulator.processed_scaler_dict, _detailed=False, nb_intervals=-1, interval_offset=-1)

    result_informed = cpp_soft_info.decode_IQ_shots_flat_informed(simulator.stim_circ.detector_error_model(decompose_errors=False), IQ_data,
                                           rounds, int(LOGICAL), resets, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                           p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=False, _ntnn_edges = True)

    return result_grid, result_informed


In [5]:
import numpy as np

distances = np.linspace(5, 55, 15)
distances = distances[11:]
distances

array([44.28571429, 47.85714286, 51.42857143, 55.        ])

In [6]:
import numpy as np
from tqdm import tqdm

shots = int(10e6) 
p_meas = -1

i_10 = True
i_20 = True
i_30 = True
i_40 = True
i_55 = True
# i_30 = False
# i_40 = False
# i_55 = False

file_str = f'../results/soft_vs_informed_simul_WITH_RESETS_v2.json'

for i in tqdm(distances, desc='Simulating for distance'):

    # Save results
    with open(file_str, 'r') as f:
        data = json.load(f)


    i = int(i)
    _redecode = False

    if i>=10 and i_10 == True:
        distance = 10
        rounds = 10
        _redecode = True
        i_10 = False

    if i>=20 and i_20 == True:  
        distance = 20
        rounds = 20
        _redecode = True
        i_20 = False

    if i >= 30 and i_30 == True:
        distance = 30
        rounds = 35
        _redecode = True
        i_30 = False
    
    if i >=40 and i_40:
        distance = i
        rounds = 35
        _redecode = True
        i_40 = False

    if i>=55 and i_55 == True:
        distance = i
        rounds = 35
        _redecode = True
        i_55 = False


    if _redecode:
        # Decode
        # result_grid, result_informed = func(distance, rounds, shots = shots, p_meas=p_meas)

        if str((distance, rounds)) not in data.keys():
            result_grid, result_informed = func(distance, rounds, shots = shots, p_meas=p_meas, resets=_RESETS)
            data[str((distance, rounds))] = {'grid': result_grid.num_errors, 'informed': result_informed.num_errors, 'shots': shots, 'p_meas': p_meas, 'resets': _RESETS}

        with open(file_str, 'w') as f:
            json.dump(data, f, indent=4)
        


   
    distance = i
    rounds = i

    
    if str((distance, rounds)) not in data.keys():
        # Decode
        result_grid, result_informed = func(distance, rounds, shots = shots, p_meas=p_meas, resets=_RESETS)
        data[str((distance, rounds))] = {'grid': result_grid.num_errors, 'informed': result_informed.num_errors, 'shots': shots, 'p_meas': p_meas, 'resets': _RESETS}
    
    with open(file_str, 'w') as f:
        json.dump(data, f, indent=4)


Simulating for distance:   0%|          | 0/4 [00:00<?, ?it/s]

: 